In [2]:
import numpy as np
#import jax.numpy as jnp

data = np.loadtxt("/var/home/luka/proj/Papilonidae_dataset_v2/Papilionidae_aligned_new.txt",delimiter="\t").reshape((2240, 200))


In [3]:
from ete3 import Tree

# Load the tree and get the leaf order in inorder traversal
ptree = Tree("/var/home/luka/proj/Papilonidae_dataset_v2/papilionidae_tree.txt", format=1)
order = ptree.get_leaf_names()  # returns in-order leaves (same as pre-order when just looking at leaves)


In [4]:
import pandas as pd

# Load categories
categories = pd.read_csv("/var/home/luka/proj/Papilonidae_dataset_v2/Papilonidae_metadata_new.txt", header=None)[0]

# Create DataFrame from data
df = pd.DataFrame(data.reshape(2240, -1))
df['category'] = pd.Categorical(categories, categories=categories.unique())

# Group by category and calculate means
means = df.groupby('category', observed=True).mean()

# Add order column based on the tree order
df['order'] = pd.Categorical(df['category'], categories=order, ordered=True)


# Reorder means DataFrame based on the order column
means = means.reset_index()
means['order'] = pd.Categorical(means['category'], categories=order, ordered=True)
means = means.sort_values('order')

# Drop unnecessary columns and convert to numpy array
X = means.drop(columns=['category', 'order']).values

N, d = X.shape

In [6]:
D = np.zeros((N*d, d))

# Create an array of indices
i_indices = np.arange(N*d)
j_indices = np.arange(d)

# Use broadcasting to create a mask
mask = (j_indices[:, None] * N <= i_indices) & (i_indices < (j_indices[:, None] + 1) * N)

D[mask.T] = 1.0


In [7]:
preorder = [n for n in ptree.traverse("preorder")]
M = len(preorder)   # number of nodes in tree (including internal)

inds = np.zeros(N, dtype=int)
inds_r = np.negative(np.ones(M, dtype=int))

dists = np.zeros((M))   # dists to root

def add_to_descendants(node, ni):
    m = len(node.get_descendants())+1
    dists[ni:ni+m] += node.dist

j = 0
for i in range(1,M):
    n = preorder[i]
    add_to_descendants(n,i)

    if n.name[0] != 'Q':
        inds[j] = i
        inds_r[i] = j
        j += 1

In [37]:
for m in range(M):
    print(f'{m:<10}{dists[m]:<20f}{preorder[m].name}')

0         0.000000            QS1803
1         0.171100            Baronia_brevicornis
2         0.045270            QS1804
3         0.110570            QS1807
4         0.195290            Iphiclides_podalirius
5         0.123720            QS1808
6         0.184680            QS1810
7         0.253510            Graphium_evemon
8         0.198370            QS1811
9         0.253030            Graphium_sarpedon
10        0.250250            Graphium_agamemnon
11        0.215990            Protographium_marcellus
12        0.065650            QS1818
13        0.091370            QS1819
14        0.151780            QS1820
15        0.196220            Hypermnestra_helios
16        0.179380            QS1821
17        0.197300            Parnassius_orleans
18        0.210610            Parnassius_honrathi
19        0.111090            QS1825
20        0.144970            QS1826
21        0.180780            Archon_apollinus
22        0.181430            Luehdorfia_puziloi
23        0.

In [9]:

# Step 1: Euler Tour and Depth Array
def euler_tour(node):
    global euler, depth, first_occurrence, current_depth
    euler = []
    depth = []
    first_occurrence = {}
    current_depth = 0

    def euler_tour_rec(node):
        global euler, depth, first_occurrence, current_depth
        euler.append(node)
        depth.append(current_depth)
        if node not in first_occurrence:
            first_occurrence[node] = len(euler) - 1
            #print(first_occurrence)
        current_depth += 1
        for child in node.children:
            euler_tour_rec(child)
            euler.append(node)
            depth.append(current_depth)
        current_depth -= 1

    euler_tour_rec(node)
    #return euler, depth, first_occurrence

# Step 2: Build Segment Tree for RMQ
def build_segment_tree():
    global st, depth, n
    n = len(depth)
    st = [0] * (2 * n)
    for i in range(n):
        st[n + i] = i
    for i in range(n - 1, 0, -1):
        if depth[st[i * 2]] < depth[st[i * 2 + 1]]:
            st[i] = st[i * 2]
        else:
            st[i] = st[i * 2 + 1]
    #return st

def rmq(l, r):
    global st, depth, n
    res = l
    while l < r:
        if l % 2:
            if depth[st[l]] < depth[st[res]]:
                res = st[l]
            l += 1
        if r % 2:
            r -= 1
            if depth[st[r]] < depth[st[res]]:
                res = st[r]
        l //= 2
        r //= 2
    return res

# Step 3: LCA Query
def lca(u, v):
    global first_occurrence, euler, depth, n, st
    if first_occurrence[u] > first_occurrence[v]:
        u, v = v, u
    l = first_occurrence[u]
    r = first_occurrence[v]
    #print(f"l: {l}, r: {r}")  # Debugging statement
    index = rmq(l+n, r+n+1) - n
    #print(f"index: {index}")  # Debugging statement
    return euler[index]


In [19]:
global euler, depth, first_occurrence, current_depth
euler = depth = first_occurrence = current_depth = None

leaves = ptree.get_leaves()
#euler, depth, first_occurrence = euler_tour(ptree)
#st = build_segment_tree(depth)

euler_tour(ptree)
build_segment_tree()
#print(depth)

lca_matrix = np.zeros((N, N), dtype=int)
lca2r_dists = np.zeros((N, N))

for i in range(N):
    for j in range(i, N):
        ancestor = lca(leaves[i], leaves[j])
        lca_ij = preorder.index(ancestor)
        lca_matrix[i, j] = lca_matrix[j, i] = lca_ij
        lca2r_dists[i, j] = lca2r_dists[j, i] = dists[lca_ij]
        #if i == j:
        #    print(i, lca_ij, inds_r[i])


np.max(lca_matrix), M
#for i in range(N):
#    print(i, '\t',lca_matrix[i, i],'\t',preorder.index(preorder[i]))


(94, 95)

In [30]:
for n in range(N):
    node = leaves[n]
    print(f'{n:<10}{node.name == lca(node,node).name}')
    #print(f'{n:<10}{lca_matrix[n,n]}')

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
30        True
31        True
32        True
33        True
34        True
35        True
36        True
37        True
38        True
39        True
40        True
41        True
42        True
43        True
44        True
45        True
46        True
47        True


In [133]:
for i in range(M):
    name = preorder[i].name
    j = inds_r[i]
    extra = "" if j < 0 else j
    print(f'{name:<25}{dists[i]:<10f}{extra}')

QS1803                   0.000000  
Baronia_brevicornis      0.171100  0
QS1804                   0.045270  
QS1807                   0.110570  
Iphiclides_podalirius    0.195290  1
QS1808                   0.123720  
QS1810                   0.184680  
Graphium_evemon          0.253510  2
QS1811                   0.198370  
Graphium_sarpedon        0.253030  3
Graphium_agamemnon       0.250250  4
Protographium_marcellus  0.215990  5
QS1818                   0.065650  
QS1819                   0.091370  
QS1820                   0.151780  
Hypermnestra_helios      0.196220  6
QS1821                   0.179380  
Parnassius_orleans       0.197300  7
Parnassius_honrathi      0.210610  8
QS1825                   0.111090  
QS1826                   0.144970  
Archon_apollinus         0.180780  9
Luehdorfia_puziloi       0.181430  10
QS1828                   0.142020  
Sericinus_montela        0.231710  11
QS1831                   0.172410  
Zerynthia_polyxena       0.196490  12
Allancastria

In [87]:

def ecov(a,b):
    na,nb = order[a],order[b]
    if a == b:
        mrca = a
    else:
        mrca = ptree.get_common_ancestor(na,nb)

    dist = dists[mrca]

    return dist

Cov = np.empty((N,N))

for i in range(N):
    for j in range(N):
        Cov[i,j] = ecov(i,j)




In [60]:
ls = ptree.get_leaves()
#print(ls[0], type(ls))
i = 0
for n in ptree.traverse("preorder"):
    na = n.name
    m = ls[i]
    if na[0] != 'Q':
        print(na)
        #lna = ls[i]
        #print(na,'\t',rna,'\t',na == rna)
        #print(f'{na:_<30}{len(n.get_ancestors())}')#{lna:_<40}{na==lna}')
        i += 1


Baronia_brevicornis
Iphiclides_podalirius
Graphium_evemon
Graphium_sarpedon
Graphium_agamemnon
Protographium_marcellus
Hypermnestra_helios
Parnassius_orleans
Parnassius_honrathi
Archon_apollinus
Luehdorfia_puziloi
Sericinus_montela
Zerynthia_polyxena
Allancastria_cerisyi
Teinopalpus_imperialis
Battus_polydamas
Battus_belus
Pharmacophagus_antenor
Trogonoptera_brookiana
Troides_rhadamantus
Ornithoptera_richmondia
Euryades_corethrus
Parides_agavus
Parides_photinus
Parides_eurimedes
Cressida_cressida
Byasa_alcinous
Atrophaneura_semperi
Atrophaneura_dixoni
Pachliopta_aristolochiae
Pachliopta_kotzebuea
Losaria_coon
Papilio_aristodemus
Papilio_thoas
Papilio_cresphontes
Papilio_slateri
Papilio_glaucus
Papilio_troilus
Papilio_gigon
Papilio_xuthus
Papilio_polyxenes
Papilio_zelicaon
Papilio_deiphobus
Papilio_protenor
Papilio_polytes
Papilio_phestus
Papilio_ambrax
Meandrusa_sciron


In [91]:
N

48

In [61]:
v1 = jnp.ones(N)
evoCov_inv = jnp.linalg.inv(Cov)
tmp = v1.T @ evoCov_inv
mle_r = ((tmp @ v1) **-1) * (tmp @ X)
assert mle_r.shape==(d,)

tmp = X - mle_r.T
mle_R = (((N - 1) ** -1) * tmp.T) @ evoCov_inv @ tmp
assert mle_R.shape==(d,d)

In [62]:
X_mean = jnp.mean(X,axis=0)
evals, evecs = jnp.linalg.eigh(Cov)
X_cent = X - X_mean[None,:]

def ppca_recon(k=2):
    V_k = evecs[:, -k:]
    #print(X_cent.shape,V_k.shape)
    X_reduced = X_cent.T @ V_k
    #print(X_reduced.shape)
    X_reconstructed = X_reduced @ V_k.T + X_mean[:,None]
    #print(X_reconstructed.shape)
    return X_reconstructed

runtime modul

In [64]:
for k in range(1,N+1):
    ppca_recon(k)